In [1]:
import pandas as pd
import os
import itertools
import networkx as nx
import random
from collections import deque
import multiprocessing as mp
import pickle


# set wd
# os.chdir('/Users/mac/Library/CloudStorage/OneDrive-McGillUniversity/Work/Projects/Gender and brokerage/WomenLeaders_SocialNetworks')
os.chdir(r'C:\Users\nnguye79\OneDrive - McGill University\Work\Projects\Gender and brokerage\WomenLeaders_SocialNetworks')

In [9]:
movies_2000_onward = pd.read_csv('movies_2000_onward.csv')

In [10]:
movies_2000_onward.head()

,tconst,startYear,genres
0,tt0011801,2019.0,"Action,Crime"
1,tt0013274,2021.0,Documentary
2,tt0015414,2000.0,\N
3,tt0028248,2022.0,Musical
4,tt0035423,2001.0,"Comedy,Fantasy,Romance"


In [13]:
# Load title.principals.tsv.gz
url_principals = 'https://datasets.imdbws.com/title.principals.tsv.gz'
df_principals = pd.read_csv(url_principals, sep='\t', compression='gzip', encoding='utf-8', low_memory=False)

# Filter for creative roles
creative_roles = ['director', 'producer', 'writer', 'editor', 'cinematographer', 'production_designer', 'composer']
df_principals = df_principals[df_principals['category'].isin(creative_roles)]

# Merge the datasets
# df_collaborations has 'tconst' for movie ID, 'nconst' for person ID, and 'startYear' for the movie year
df_collaborations = df_principals[['tconst', 'nconst', 'category']].merge(movies_2000_onward[['tconst', 'startYear']], on='tconst', how='inner')
df_collaborations.to_csv('df_collaborations.csv' , index=False)

In [2]:
df_collaborations = pd.read_csv('df_collaborations.csv')

df_collaborations.nunique()

tconst       296352
nconst       571566
category          7
startYear        24
dtype: int64

In [4]:
df_collaborations.head()

,tconst,nconst,category,startYear
0,tt0011801,nm0483944,writer,2019.0
1,tt0011801,nm1773808,cinematographer,2019.0
2,tt0013274,nm0412842,director,2021.0
3,tt0013274,nm0895048,director,2021.0
4,tt0013274,nm13054604,producer,2021.0


In [20]:
random.sample(df_collaborations['tconst'].unique().tolist(), 3)

['tt9664506', 'tt13130596', 'tt15856752']

In [21]:
toy = df_collaborations[df_collaborations['tconst'].isin(['tt9664506', 'tt13130596', 'tt15856752'])]

In [22]:
toy

,tconst,nconst,category,startYear
319017,tt13130596,nm4021775,director,2023.0
319018,tt13130596,nm9442886,producer,2023.0
319019,tt13130596,nm7344462,producer,2023.0
319020,tt13130596,nm6871943,producer,2023.0
465595,tt15856752,nm0759038,director,2021.0
465596,tt15856752,nm1232403,composer,2021.0
1202597,tt9664506,nm2348718,director,2019.0
1202598,tt9664506,nm7684256,writer,2019.0
1202599,tt9664506,nm1930572,composer,2019.0


In [27]:
# create collaboration pairs
def create_pairs(group):
    return pd.DataFrame(itertools.combinations(group['nconst'], 2), columns=['person1', 'person2'])

In [29]:
toy.groupby(['tconst', 'startYear']).apply(create_pairs)

/var/folders/1j/csv118695x53qkzhd9p2tgcc0000gn/T/ipykernel_1071/149914025.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  toy.groupby(['tconst', 'startYear']).apply(create_pairs)


person1    person2
tconst     startYear                        
tt13130596 2023.0    0  nm4021775  nm9442886
                     1  nm4021775  nm7344462
                     2  nm4021775  nm6871943
                     3  nm9442886  nm7344462
                     4  nm9442886  nm6871943
                     5  nm7344462  nm6871943
tt15856752 2021.0    0  nm0759038  nm1232403
tt9664506  2019.0    0  nm2348718  nm7684256
                     1  nm2348718  nm1930572
                     2  nm7684256  nm1930572

In [30]:
toy.groupby(['tconst', 'startYear']).apply(create_pairs).reset_index()

/var/folders/1j/csv118695x53qkzhd9p2tgcc0000gn/T/ipykernel_1071/3409082432.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  toy.groupby(['tconst', 'startYear']).apply(create_pairs).reset_index()


,tconst,startYear,level_2,person1,person2
0,tt13130596,2023.0,0,nm4021775,nm9442886
1,tt13130596,2023.0,1,nm4021775,nm7344462
2,tt13130596,2023.0,2,nm4021775,nm6871943
3,tt13130596,2023.0,3,nm9442886,nm7344462
4,tt13130596,2023.0,4,nm9442886,nm6871943
5,tt13130596,2023.0,5,nm7344462,nm6871943
6,tt15856752,2021.0,0,nm0759038,nm1232403
7,tt9664506,2019.0,0,nm2348718,nm7684256
8,tt9664506,2019.0,1,nm2348718,nm1930572
9,tt9664506,2019.0,2,nm7684256,nm1930572


In [31]:
toy.groupby(['tconst', 'startYear']).apply(create_pairs).reset_index().drop('level_2', axis=1)

/var/folders/1j/csv118695x53qkzhd9p2tgcc0000gn/T/ipykernel_1071/2236794962.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  toy.groupby(['tconst', 'startYear']).apply(create_pairs).reset_index().drop('level_2', axis=1)


,tconst,startYear,person1,person2
0,tt13130596,2023.0,nm4021775,nm9442886
1,tt13130596,2023.0,nm4021775,nm7344462
2,tt13130596,2023.0,nm4021775,nm6871943
3,tt13130596,2023.0,nm9442886,nm7344462
4,tt13130596,2023.0,nm9442886,nm6871943
5,tt13130596,2023.0,nm7344462,nm6871943
6,tt15856752,2021.0,nm0759038,nm1232403
7,tt9664506,2019.0,nm2348718,nm7684256
8,tt9664506,2019.0,nm2348718,nm1930572
9,tt9664506,2019.0,nm7684256,nm1930572


In [49]:
random_movies = df_collaborations[df_collaborations['startYear'].isin([2018, 2019, 2020, 2021, 2022, 2023])].groupby('startYear')['tconst'].apply(lambda x: random.sample(x.tolist(), min(2, len(x)))).explode().tolist()
toy = df_collaborations[df_collaborations['tconst'].isin(random_movies)].sort_values(by=['startYear', 'tconst'])

In [50]:
toy

,tconst,nconst,category,startYear
942832,tt4701724,nm0520485,producer,2018.0
942833,tt4701724,nm0661910,director,2018.0
942834,tt4701724,nm0123666,writer,2018.0
942835,tt4701724,nm2664949,writer,2018.0
942836,tt4701724,nm0640976,writer,2018.0
942837,tt4701724,nm1485380,producer,2018.0
1158932,tt8544336,nm9867435,director,2018.0
1158933,tt8544336,nm9901624,director,2018.0
1158934,tt8544336,nm7540376,writer,2018.0
1158935,tt8544336,nm9293741,producer,2018.0


In [55]:
# create collaboration pairs
def create_pairs(group):
    return pd.DataFrame(itertools.combinations(group['nconst'], 2), columns=['person1', 'person2'])

toy = toy.groupby(['tconst', 'startYear']).apply(create_pairs).reset_index().drop('level_2', axis=1)

/var/folders/1j/csv118695x53qkzhd9p2tgcc0000gn/T/ipykernel_1071/2835858188.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  toy = toy.groupby(['tconst', 'startYear']).apply(create_pairs).reset_index().drop('level_2', axis=1)


In [83]:
toy

,tconst,startYear,person1,person2
0,tt10671436,2019.0,nm4431035,nm3221779
1,tt10671436,2019.0,nm4431035,nm6465550
2,tt10671436,2019.0,nm4431035,nm10840333
3,tt10671436,2019.0,nm4431035,nm10011769
4,tt10671436,2019.0,nm4431035,nm6482598
...,...,...,...,...
105,tt9600550,2019.0,nm3565330,nm1415396
106,tt9600550,2019.0,nm3565330,nm2075583
107,tt9600550,2019.0,nm6314036,nm1415396
108,tt9600550,2019.0,nm6314036,nm2075583


In [98]:
toy[toy['startYear'] == 2018]


,tconst,startYear,person1,person2
67,tt4701724,2018.0,nm0520485,nm0661910
68,tt4701724,2018.0,nm0520485,nm0123666
69,tt4701724,2018.0,nm0520485,nm2664949
70,tt4701724,2018.0,nm0520485,nm0640976
71,tt4701724,2018.0,nm0520485,nm1485380
72,tt4701724,2018.0,nm0661910,nm0123666
73,tt4701724,2018.0,nm0661910,nm2664949
74,tt4701724,2018.0,nm0661910,nm0640976
75,tt4701724,2018.0,nm0661910,nm1485380
76,tt4701724,2018.0,nm0123666,nm2664949


In [104]:
edges_by_year = {year: group[['person1', 'person2']].values for year, group in toy.groupby('startYear')}

In [113]:
def calculate_social_capital(edges_by_year, year):
    rolling_edges = deque()
    for y in range(year - 3, year):
        edges = edges_by_year.get(y, [])
        rolling_edges.extend(edges)
    
    G = nx.Graph()
    G.add_edges_from(rolling_edges)
    
    centrality_scores = {
        'degree_centrality': nx.degree_centrality(G),
        'betweenness_centrality': nx.betweenness_centrality(G),
        'closeness_centrality': nx.closeness_centrality(G),
        'eigenvector_centrality': nx.eigenvector_centrality(G, max_iter=1000),  # Increased max_iter for convergence
        'constraint': nx.constraint(G),
        'effective_size': nx.effective_size(G)
    }
    return centrality_scores

In [114]:
calculate_social_capital(edges_by_year, 2023)

{'degree_centrality': {'nm12016644': 0.17391304347826086,
  'nm7334748': 0.17391304347826086,
  'nm11147604': 0.17391304347826086,
  'nm12005539': 0.17391304347826086,
  'nm6442094': 0.17391304347826086,
  'nm13296042': 0.17391304347826086,
  'nm14589356': 0.17391304347826086,
  'nm12531463': 0.17391304347826086,
  'nm10055037': 0.17391304347826086,
  'nm13296045': 0.17391304347826086,
  'nm5167561': 0.17391304347826086,
  'nm5405493': 0.17391304347826086,
  'nm9272600': 0.17391304347826086,
  'nm12174545': 0.17391304347826086,
  'nm2299576': 0.17391304347826086,
  'nm6908362': 0.08695652173913043,
  'nm10682718': 0.08695652173913043,
  'nm8514200': 0.08695652173913043,
  'nm0649629': 0.08695652173913043,
  'nm11947097': 0.08695652173913043,
  'nm0310602': 0.08695652173913043,
  'nm14644865': 0.08695652173913043,
  'nm3157059': 0.08695652173913043,
  'nm14644866': 0.08695652173913043},
 'betweenness_centrality': {'nm12016644': 0.0,
  'nm7334748': 0.0,
  'nm11147604': 0.0,
  'nm12005539

In [84]:
toy_1 = toy[toy['startYear'].isin([2020, 2021, 2022])]

In [94]:
collaboration_count = pd.concat([toy_1['person1'], toy_1['person2']]).value_counts()
collaboration_count / (24 - 1)

nm12016644    0.173913
nm13296042    0.173913
nm13296045    0.173913
nm2299576     0.173913
nm6442094     0.173913
nm10055037    0.173913
nm7334748     0.173913
nm14589356    0.173913
nm12531463    0.173913
nm12174545    0.173913
nm9272600     0.173913
nm5405493     0.173913
nm5167561     0.173913
nm12005539    0.173913
nm11147604    0.173913
nm11947097    0.086957
nm0649629     0.086957
nm14644865    0.086957
nm3157059     0.086957
nm6908362     0.086957
nm10682718    0.086957
nm0310602     0.086957
nm14644866    0.086957
nm8514200     0.086957
Name: count, dtype: float64

In [115]:
centrality_scores_by_year = {}

for year in range(2021, 2024):  # From 2003 to 2013 inclusive
    centrality_scores = calculate_social_capital(edges_by_year, year)
    centrality_scores_by_year[year] = centrality_scores


In [116]:
centrality_scores_by_year

{2021: {'degree_centrality': {'nm0520485': 0.15625,
   'nm0661910': 0.15625,
   'nm0123666': 0.15625,
   'nm2664949': 0.15625,
   'nm0640976': 0.15625,
   'nm1485380': 0.15625,
   'nm9867435': 0.125,
   'nm9901624': 0.125,
   'nm7540376': 0.125,
   'nm9293741': 0.125,
   'nm9957690': 0.125,
   'nm4431035': 0.15625,
   'nm3221779': 0.15625,
   'nm6465550': 0.15625,
   'nm10840333': 0.15625,
   'nm10011769': 0.15625,
   'nm6482598': 0.15625,
   'nm3850892': 0.15625,
   'nm4022741': 0.15625,
   'nm3565330': 0.15625,
   'nm6314036': 0.15625,
   'nm1415396': 0.15625,
   'nm2075583': 0.15625,
   'nm12016644': 0.125,
   'nm7334748': 0.125,
   'nm11147604': 0.125,
   'nm12005539': 0.125,
   'nm6442094': 0.125,
   'nm13296042': 0.125,
   'nm14589356': 0.125,
   'nm12531463': 0.125,
   'nm10055037': 0.125,
   'nm13296045': 0.125},
  'betweenness_centrality': {'nm0520485': 0.0,
   'nm0661910': 0.0,
   'nm0123666': 0.0,
   'nm2664949': 0.0,
   'nm0640976': 0.0,
   'nm1485380': 0.0,
   'nm9867435':

In [ ]:
# create collaboration pairs
def create_pairs(group):
    return pd.DataFrame(itertools.combinations(group['nconst'], 2), columns=['person1', 'person2'])

df_collaborations = df_collaborations.groupby(['tconst', 'startYear']).apply(create_pairs).reset_index().drop('level_2', axis=1)
edges_by_year = {year: group[['person1', 'person2']].values for year, group in df_collaborations.groupby('startYear')}

def calculate_social_capital(edges_by_year, year):
    rolling_edges = deque()
    for y in range(year - 3, year):
        edges = edges_by_year.get(y, [])
        rolling_edges.extend(edges)
    
    G = nx.Graph()
    G.add_edges_from(rolling_edges)
    
    centrality_scores = {
        'degree_centrality': nx.degree_centrality(G),
        'betweenness_centrality': nx.betweenness_centrality(G),
        'closeness_centrality': nx.closeness_centrality(G),
        'eigenvector_centrality': nx.eigenvector_centrality(G, max_iter=1000),  # Increased max_iter for convergence
        'constraint': nx.constraint(G),
        'effective_size': nx.effective_size(G)
    }
    return centrality_scores

centrality_scores_by_year = {}

for year in range(2003, 2014):  # From 2003 to 2013 inclusive
    centrality_scores = calculate_social_capital(edges_by_year, year)
    centrality_scores_by_year[year] = centrality_scores

# Save as a pickle file
with open('centrality_scores.pkl', 'wb') as f:
    pickle.dump(centrality_scores_by_year, f)
